In [1]:
%cd ../

/home/hoanghu/projects/Food-Waste-Optimization/experiments_hoangle


In [2]:
import numpy as np
import pandas as pd
from itertools import combinations, product

from utils import Paths

# Load related processed data

In [3]:
wastes = pd.read_excel(Paths.pred_biowaste())

wastes.head()

,dish,waste
0,Aurajuusto-pinaattilasagnettea,0.066898
1,BBQ-Broilerikastiketta,0.060485
2,Bangladeshilainen linssipata,0.059193
3,Bataatti-maapähkinäkeitto,0.010000
4,Bataattipihvit,0.050015


In [4]:
co2 = pd.read_excel(Paths.pred_co2())

co2.head()

,restaurant,category,dish,co2
0,Chemicum,chicken,BBQ-Broilerikastiketta,0.57
1,Chemicum,chicken,Broileria appelsiini-currykastikkeessa,0.64
2,Chemicum,chicken,Broileria pekonikastikkeessa,0.99
3,Chemicum,chicken,Broileria pestokastikkeessa,0.54
4,Chemicum,chicken,Broilerinkoipea,1.28


In [5]:
pieces_whole = pd.read_excel(Paths.pred_n_pcs_whole(), parse_dates=['date'])

pieces_whole.head()

,date,pcs,restaurant
0,2024-09-02,728.91,Chemicum
1,2024-09-03,729.82,Chemicum
2,2024-09-04,707.55,Chemicum
3,2024-09-05,660.46,Chemicum
4,2024-09-06,678.02,Chemicum


In [6]:
pieces_dishes = pd.read_excel(Paths.pred_n_pcs_per_dish(), parse_dates=['date'])

pieces_dishes.head()

,restaurant,category,dish,date,pcs
0,Chemicum,chicken,BBQ-Broilerikastiketta,2024-09-02,202
1,Chemicum,chicken,Broileria appelsiini-currykastikkeessa,2024-09-02,76
2,Chemicum,chicken,Broileria pekonikastikkeessa,2024-09-02,129
3,Chemicum,chicken,Broileria pestokastikkeessa,2024-09-02,78
4,Chemicum,chicken,Broilerinkoipea,2024-09-02,94


# Create all possible combinations

In [25]:
N_VEGAN_PER_DAY = 2
N_OTHER_PER_DAY = 2

N_LIMIT = 2_000_000     # This threshold limits the number of considering menus

restaurant = "Physicum"
date = pd.to_datetime('2024-09-06')

## Create dataframe legitimate menus (2 vegans per day)

In [26]:
dishes_vegan = pieces_dishes[
    (pieces_dishes['category'] == 'vegan')
    & (pieces_dishes['restaurant'] == restaurant)
]['dish'].unique()

dishes_other = pieces_dishes[
    (pieces_dishes['category'] != 'vegan')
    & (pieces_dishes['restaurant'] == restaurant)
]['dish'].unique()

In [27]:
list_menus = product(combinations(dishes_vegan, N_VEGAN_PER_DAY), combinations(dishes_other, N_OTHER_PER_DAY))

menus = pd.DataFrame.from_records([
    {
        'dish_1': menu[0][0],
        'dish_2': menu[0][1],
        'dish_3': menu[1][0],
        'dish_4': menu[1][1],
    }
    for menu in list_menus
])

# This line is used to limit the the number of considering menus
menus = menus.sample(min([N_LIMIT, len(menus)])).reset_index(drop=True)

menus.head()

,dish_1,dish_2,dish_3,dish_4
0,Lounaspatonki,Meksikolainen nyhtökaurasalaat,Panini poro,Vuohenjuustosalaattia
1,Lounaspatonki tofu,Panini Bombay,Grillattu kana ohutleipärulla,Tomaatti-mozzarellasalaatti
2,Panini,Panini Bombay,Grillattu kana ohutleipärulla,Panini poro
3,Lounaspatonki tofu,Panini,Kinkku-aura ohutleipärulla,Vuohenjuustosalaattia
4,Lounaspatonki,Meksikolainen nyhtökaurasalaat,Broileri-nacho-salaattia,Grillattu kana ohutleipärulla


## Merge with other tables

In [28]:
# Get 'waste' for all dishes
menus = (
    menus
    .merge(wastes, left_on='dish_1', right_on='dish', how='left').drop(columns='dish').rename(columns={'waste': 'waste_1'})
    .merge(wastes, left_on='dish_2', right_on='dish', how='left').drop(columns='dish').rename(columns={'waste': 'waste_2'})
    .merge(wastes, left_on='dish_3', right_on='dish', how='left').drop(columns='dish').rename(columns={'waste': 'waste_3'})
    .merge(wastes, left_on='dish_4', right_on='dish', how='left').drop(columns='dish').rename(columns={'waste': 'waste_4'})
)

# Get 'co2' for all dishes
cols_redundant = ['dish', 'category', 'restaurant']
co2_restaurant = co2[co2['restaurant'] == restaurant].groupby(['dish']).first().reset_index(drop=False)
menus = (
    menus
    .merge(co2_restaurant, left_on='dish_1', right_on='dish', how='left').drop(columns=cols_redundant).rename(columns={'co2': 'co2_1'})
    .merge(co2_restaurant, left_on='dish_2', right_on='dish', how='left').drop(columns=cols_redundant).rename(columns={'co2': 'co2_2'})
    .merge(co2_restaurant, left_on='dish_3', right_on='dish', how='left').drop(columns=cols_redundant).rename(columns={'co2': 'co2_3'})
    .merge(co2_restaurant, left_on='dish_4', right_on='dish', how='left').drop(columns=cols_redundant).rename(columns={'co2': 'co2_4'})
)

# Get no. predicted sold pieces for all dishes
cols_redundant = ['dish', 'category', 'restaurant', 'date']

pieces_dishes_restaurant = pieces_dishes[(pieces_dishes['date'] == date) & (pieces_dishes['restaurant'] == restaurant)].groupby(['dish']).first().reset_index(drop=False)
menus = (
    menus
    .merge(pieces_dishes_restaurant, left_on='dish_1', right_on='dish', how='left').drop(columns=cols_redundant).rename(columns={'pcs': 'pcs_1'})
    .merge(pieces_dishes_restaurant, left_on='dish_2', right_on='dish', how='left').drop(columns=cols_redundant).rename(columns={'pcs': 'pcs_2'})
    .merge(pieces_dishes_restaurant, left_on='dish_3', right_on='dish', how='left').drop(columns=cols_redundant).rename(columns={'pcs': 'pcs_3'})
    .merge(pieces_dishes_restaurant, left_on='dish_4', right_on='dish', how='left').drop(columns=cols_redundant).rename(columns={'pcs': 'pcs_4'})
)

# Filter menus whose CO2 or waste of any dish is missng
menus = menus[~menus.isna().any(axis=1)]
menus.head()

,dish_1,dish_2,dish_3,dish_4,waste_1,waste_2,waste_3,waste_4,co2_1,co2_2,co2_3,co2_4,pcs_1,pcs_2,pcs_3,pcs_4
0,Lounaspatonki,Meksikolainen nyhtökaurasalaat,Panini poro,Vuohenjuustosalaattia,0.181925,0.400921,0.010000,0.519063,0.53,0.57,0.91,1.32,9,12,8,5
6,Lounaspatonki,Lounaspatonki tofu,Nyhtöporsassalaatti,Kreikkalainen salaatti,0.181925,0.074023,1.232735,0.306367,0.53,0.41,0.70,1.14,9,5,5,6
11,Meksikolainen nyhtökaurasalaat,Panini Bombay,Panini,Tonnikalasalaatti,0.400921,0.010000,0.010000,0.319784,0.57,0.43,0.91,0.84,12,6,27,6
13,Lounaspatonki tofu,Meksikolainen nyhtökaurasalaat,Panini poro,Tomaatti-mozzarellasalaatti,0.074023,0.400921,0.010000,0.255910,0.41,0.57,0.91,1.13,5,12,8,5
14,Lounaspatonki tofu,Meksikolainen nyhtökaurasalaat,Broileri-nacho-salaattia,Panini poro,0.074023,0.400921,0.315456,0.010000,0.41,0.57,0.80,0.91,5,12,5,8


## Calculate terms

In [29]:
THETA_CO2 = 0.5
THETA_WASTE = 0.04
ALPHA_PCS = 2
ALPHA_CO2 = 1
ALPHA_WASTE = 1

In [30]:
n_pieces_whole = pieces_whole[(pieces_whole['restaurant'] == restaurant) & (pieces_whole['date'] == date)]['pcs'].item()

menus['total_co2'] = (
    menus['co2_1'] * menus['pcs_1']
    + menus['co2_2'] * menus['pcs_2']
    + menus['co2_3'] * menus['pcs_3']
    + menus['co2_4'] * menus['pcs_4']
)

menus['total_waste'] = (
      menus['waste_1'] * menus['pcs_1']
    + menus['waste_2'] * menus['pcs_2']
    + menus['waste_3'] * menus['pcs_3']
    + menus['waste_4'] * menus['pcs_4']
)

menus['total_pcs_from_dishes'] = menus['pcs_1'] + menus['pcs_2'] + menus['pcs_3'] + menus['pcs_4']

menus['co2_per_customer'] = menus['total_co2'] / menus['total_pcs_from_dishes']
menus['waste_per_customer'] = menus['total_waste'] / menus['total_pcs_from_dishes']

# Calculate fitness value

menus['fitness'] = (
    ALPHA_PCS * np.abs(menus['total_pcs_from_dishes'] / n_pieces_whole - 1) 
    + ALPHA_CO2 * menus['co2_per_customer'] / THETA_CO2 
    + ALPHA_WASTE * menus['waste_per_customer'] / THETA_WASTE
)

# Remove redudant columns
cols = [
    'dish_1',
    'dish_2',
    'dish_3', 
    'dish_4',
    'total_co2', 
    'total_waste',
    'total_pcs_from_dishes',
    'co2_per_customer',
    'waste_per_customer',
    'fitness'
]

menus = menus[cols]

# Sort w.r.t 'fitness'
menus.sort_values(by='fitness', ascending=True, inplace=True)

menus.head()

,dish_1,dish_2,dish_3,dish_4,total_co2,total_waste,total_pcs_from_dishes,co2_per_customer,waste_per_customer,fitness
445,Panini,Panini Bombay,Panini,Panini poro,59.00,0.680000,68,0.867647,0.010000,2.630712
101,Lounaspatonki tofu,Panini,Panini,Panini poro,58.47,0.990116,67,0.872687,0.014778,2.780158
178,Panini,Panini Bombay,Lounaspatonki,Panini,56.49,2.237324,69,0.818696,0.032425,3.073514
55,Lounaspatonki tofu,Panini Bombay,Panini,Panini poro,36.48,0.780116,46,0.793043,0.016959,3.093728
17,Lounaspatonki,Panini,Panini,Panini poro,61.19,2.257324,71,0.861831,0.031793,3.104152
